In [32]:
!pip install opencv-python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import tensorflow as tf
import pandas as pd
import math
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [34]:
# padding='same' : 자동으로 패딩을 삽입해 입력값과 출력값의 크기를 맞춰줌
# padding='valid' : 패딩을 적용하지 않고 필터를 적용해서 출력값의 크기가 작아짐

def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            layers.Input(input_shape), # 입력 layer



            # 1st
            layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 64개의 필터, 3by3짜리 필터크기, 
            layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'), # strides=1은 옆으로 1칸씩 가는 것 
            # 두번 가중치를 주면 좀 더 곡선이 살아남
            layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
            layers.BatchNormalization(), # 값들을 정규화함.(사실 해도되고 안해도됨) 
                                        # 한 이유는 픽셀값들의 크기를 정리해줘서 속도를 빠르게하기위해
            layers.Dropout(0.5), # 과대적합하지않게 50%를 학습시킴(이 것도 해도되고 안해도됨)
                                # 나중에 실제로 값을 적용했을때 좋ㅁ 더 좋은 성능을 낼 수 있다.



            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 128개의 필터, 3by3짜리 필터크기, 
            layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'),
            # 두번 가중치를 주면 좀 더 곡선이 살아남
            layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
            layers.BatchNormalization(), # 값들을 정규화.
            layers.Dropout(0.3), # 과대적합하지않게 # 30%를 학습시킴        



            # 레이어를 두번 사용


            # FC
            layers.GlobalMaxPool2D(), # 여태까지 왔던 모든 데이터를 maxpool함
            layers.Dense(128, activation="relu"),
            layers.Dense(1, activation='sigmoid'), # 시그모이드로 판단
        ]
    )
    return model

In [35]:
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.compile( # compile함수 사용해서 설정값 설정
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics='accuracy'
)


In [36]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train',shuffle=True):
        self.batch_size = batch_size
        self.csv_path = csv_path
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self. shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
            
        self.on_epoch_end()
        
        
        
        
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac = 1).reset_index(drop = True)
        
        
        
        
    def __getitem__(self, idx):
        start = idx * self.batch_size # 만약 idx에 0번을 넣으면 0부터 시작, 1번이면 1* self.batch_size이다
        end = (idx + 1) * self.batch_size
        data = self.df.iloc[start:end] # 만약 0부터 64미만이라면
        batch_x, batch_y = self.get_data(data) # 리턴되는 값이 두개가 나옴
        return np.array(batch_x), np.array(batch_y)
    
    
    
    def get_data(self, data):
        batch_x = []
        batch_y = [] # self가 안붙었으니 이 batch_x,y는 이 메소드 안에서만 사용한다.
        
        for _, r in data.iterrows(): # data에서 넘어오는 리스트가 인덱스번호(_)는 안쓰고, 데이터(r)는 쓴다
            file_name = r['file_name']
            image = cv2.imread(f'C:/python/Jupyter/jupyter/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size)) # 이미지 크기가 다 제각각이므로 다 동일하게 맞춤
            iamge = image/ 255. # 나눠준 이유는 정규화하기 위해(나중에 cnn모델 쓸때 색상정보는 기울기로 쓰이는데 숫자가 낮아지면 속도가 빨라지니까)
            
            label = int(r['species']) - 1 # soecies보면 1로 되어있는데 9부터 시작하기위해 -1을 해준다
            
            batch_x.append(image) # 계속 쌓아줌
            batch_y.append(label)
            
        return batch_x, batch_y


In [37]:
csv_path = 'C:/python/Jupyter/jupyter/kfolds.csv'

train_generator = DataGenerator(
    batch_size = 128,
    csv_path = csv_path,
    fold = 1,
    image_size = 256,
    mode = 'train',
    shuffle = True    
)

valid_generator = DataGenerator(
    batch_size = 128,
    csv_path = csv_path,
    fold = 1,
    image_size = 256,
    mode = 'val',
    shuffle = True    
)


In [38]:
# EarlyStopping: Epoch을 많이 돌린 후 특정 시점에 멈추게 함

# monitor: EarlyStopping의 기준이 되는 값. 예) val_loss 더이상 감소되지 않을 경우 EarlyStopping 적용

# patience: Training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우 몇번 epoch을 진행할 지 설정

# mode: monitor되는 값이 최소가 되야 하는지, 최대가 되야 하는지 설정

# restore_best_weights: true로 설정하면 training이 끝난 후 model의 wieght를 monitor하고 있던 값이 가장
# 좋았을 경우의 weight로 복원. False라면 마지막 training이 끝난 후의 weight로 설정


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, mode='min', restore_best_weights=False
)

In [39]:
# ReduceLROnPlateau: 모델의 개선이 없을 경우 learning rate를 조절해 모델의 개선을 유도함

# monitor: ReduceLROnPlateau의 기준이 되는 값. 예) val_loss 더이상 감소되지 않을 경우 ReduceLROnPlateau 적용

# factor: learning rate를 얼마나 변경시킬 것인지 정하는 값. learning rate * factor

# patience: training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우 최적의 monitor값을 기준으로
# 몇 번의 epoch을 진행하고 learning rate를 조절할지의 값을 설정


reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10, mode='min', min_lr=0.0001
)

In [40]:
# ModelCheckpoint: 모델의 경로 설정

# 모델 경로를 '{epoch:02d}-{val_loss:.2f}.hdf5' 라고 하면 앞의 명시한 문자열로 파일을 저장
# 예) 01-0.12f.h5

# save_weights_only: True(weight만 저장), False(모델, layer, weight 모두 저장)

# save_best_only: True(모델의 정확도가 최고값을 갱신했을 때만 저장), False(매회 저장)


filepath = '{epoch:02d}-{val_loss:.2f}.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', save_best_only=True, save_weights_only=False, mode='min'
)

In [41]:
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=10,
    callbacks=[
        early_stopping,
        reduce_on_plateau,
        model_checkpoint
    ]
)

NotImplementedError: 